# 道路追踪-现场演示

在本示例中，我们将使用经过训练的模型来使jetBot平稳地在轨道上移动。

### 加载训练模型

假定您已经将``best_steering_model_xy.pth``按照“ train_model.ipynb”notebook中的说明下载到工作站。现在，您应该将模型文件上传到该JetBot的本notebook示例目录中。完成后，在此示例的目录中应该有一个名为``best_steering_model_xy.pth``的文件。

> 在调用下一个单元格之前，请确保文件已完全上传

执行以下代码以初始化PyTorch模型。这个和训练阶段非常类似。

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

接下来，从您上传的``best_steering_model_xy.pth``文件中加载训练后的权重。

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

当前，模型权重位于CPU内存上，执行以下代码以传输到GPU设备。

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### 创建预处理功能

现在，我们已经加载了模型，但是有一个小问题。我们训练模型的格式与相机的格式不完全匹配。为此，我们需要进行一些预处理。这涉及以下步骤：

1. 从HWC格式转换为CHW格式
2. 使用与训练期间相同的参数进行归一化（我们的相机提供[0，255]范围内的值，并训练[0，1]范围内的已加载图像，因此我们需要缩放255.0
3. 将数据从CPU内存传输到GPU内存
4. 添加批次维度

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

太棒了！现在，我们定义了预处理功能，该功能可以将图像从相机格式转换为神经网络输入格式。

现在，让我们开始显示相机。您现在应该已经对此非常熟悉。

In [ ]:
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

def my_car(camera_image):
    image = np.copy(camera_image)
    image = cv2.line(image, (100,224), (24,200), (0,255,255), 3)
    image = cv2.line(image, (124,224), (200,200), (0,255,255), 3)
    image = cv2.line(image, (106,224), (106,212), (0,255,0), 3)
    image = cv2.line(image, (118,224), (118,212), (0,255,0), 3)
    image = cv2.line(image, (106,212), (118,212), (0,255,0), 3)
    
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=my_car)

display(widgets.HBox([image_widget, target_widget]))

我们还将创建机器人实例，以驱动电机。

In [ ]:
from jetbot import Robot
#注意，可以对小车设置的速度范围为0到100

robot = Robot()

现在，我们将定义滑块来控制JetBot

> 注意：我们已经为最重要的配置初始化了滑块值，但是这些值可能不适用于您的数据集，因此请根据您的设置和环境增多或减少滑块

1. 速度控制（speed_gain_slider）：开始增加``speed_gain_slider``来启动JetBot
2. 转向增益控制（steering_gain_sloder）：如果您看到JetBot摇摆不定，则需要减小``steering_gain_slider``直到平稳为止
3. 转向偏差控制（steering_bias_slider）：如果看到JetBot偏向轨道的最右侧或最左侧，则应控制此滑块，直到JetBot开始沿中心线或轨道行进。这说明了电机偏差以及相机偏移
> 注意：您应该以较低的速度先调节上述滑块，以实现JetBot平滑的道路跟随行为。

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

接下来，让我们显示一些滑块，这些滑块将使我们看到JetBot的想法。x和y滑块将显示预测的x，y值。

转向滑块将显示我们的估算转向值。请记住，该值不是目标的实际角度，而是一个几乎成比例的值。当实际角度为时0，它将为零，并且将随实际角度增加/减少。

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-100, max=100, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

接下来，我们将创建一个函数，只要相机的值发生变化，该函数就会被调用。此函数将执行以下步骤

1. 预处理相机图像
2. 执行神经网络
3. 计算近似转向值
4. 使用比例/微分控制（PD）控制电机

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 0.5), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 0.5), 0.0)
    
execute({'new': camera.value})

酷！我们已经创建了神经网络执行功能，但是现在我们需要将其附加到相机上进行处理。

我们通过观察函数来实现。

> 警告：此代码将移动机器人！请确保您的机器人足够的运动空间并且位于你收集数据的轨迹上。道路追随者应该工作，但神经网络的性能仅取决于其训练的数据！

In [ ]:
camera.observe(execute, names='value')

太棒了！如果您的机器人已插入电源，它应该能随着每帧图像的更新产生新的命令。

现在，您可以将JetBot放在收集数据的轨迹上，并查看它是否可以跟随轨迹。

如果要停止此行为，可以通过执行以下代码来取消此回调功能。

In [ ]:
import time

camera.unobserve(execute, names='value')
camera.stop()
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### 结论

这是本示例的所有演示！希望您玩得开心，看到您的JetBot在轨道上顺畅地行驶！！！

如果您的JetBot行驶不顺利，请尝试找出失败的地方。这样做的好处是，我们可以针对这些失败情况收集更多的数据，使JetBot变得更好：)